In [10]:
from collections import OrderedDict
from torch.utils.data import Dataset
class Seq2SeqDataset(Dataset):
    def _init_(self, encode_datas, decode_datas):
        self.encode_datas = encode_datas
        self.decode_datas = decode_datas
        self.encode_vocab = self.build_vocab(encode_datas,fill_mask = ["PAD", "EOS", "UNK"])
        self.decode_vocab = self.build_vocab(decode_datas, fill_mask = ["PAD", "BOS", "EOS", "UNK"])
    def _getitem_ (self,index) :
        enc = list(self.encode_datas[index]) + ["EOS"]
        dec = ["B0S"] + list(self.decode_datas[index]) + ["EOS"]
        e = [self.encode_vocab.get(tk, self.encode_vocab['UNK']) for tk in enc]
        d = [self.decode_vocab.get(tk, self.decode_vocab['UNK']) for tk in dec]
        return e,d
    def _len_ (self):
        return len(self.encode_datas)
    def build_vocab(self, datas, fill_mask):
        #构建词汇表
        vocab = OrderedDict({ msk:idx for idx, msk in enumerate(fill_mask)})
        for item in datas:
            for token in set(item):
                vocab[token] = vocab.get(token, len(vocab))
        return vocab

In [12]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
def build_dataloader(dataset, batch_size = None, shuffle = False):
    def collate_batch(batch):
        encode_list, decode_list = [],[]
        # 解决batch_size=1时dataset返回tuple的情况
        batch = [batch] if type(batch) == tuple else batch
        for encode,decode in batch:
            processed_enc = torch.tensor(encode, dtype=torch.int64)
            processed_dec = torch.tensor(decode, dtype=torch.int64)
            encode_list.append(processed_enc)
            decode_list.append(processed_dec)
        # pad_sequence : list(tensor) > tensor matrix (auto padding)
        encode_list = pad_sequence(encode_list, batch_first=True, padding_value=0).transpose(0, 1)
        decode_list = pad_sequence(decode_list, batch_first=True, padding_value=0).transpose(0, 1)
        return encode_list, decode_list
        return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_batch)

In [13]:
def generate_square_subsequent_mask(sz):
    #torch.triu方法保留矩阵上三角部分内容，其 余部分填充为0
    #https://pytorch.org/docs/stable/generated/torch.triu.html#torch.triu
    mask = (torch.triu(torch.ones((sz, sz))) == 1).transpose(0,1)
    #float("-inf")代表负无穷，意 即是极小值
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]
    # 为解码器生成一个mask矩阵
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    # 为编码器生成一个False矩阵
    src_mask = torch.zeros((src_seq_len, src_seq_len)).type(torch.bool)
    # 生成padding矩阵，所 有填充项取值为True,未填充项取值为False
    src_padding_mask = (src == 0).transpose(0, 1)
    tgt_padding_mask = (tgt == 0).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
# 测试语料加载和转换
decode_datas, encode_datas = read_corpus(corpus_file)
dataset = Seq2SeqDataset(encode_datas, decode_datas)
# 学习率
LR = 0.0001
# 编 码器词典大小
SRC_VOCAB_SIZE = len(dataset.encode_vocab)
# 解码器词典大小
TGT_VOCAB_SIZE = len(dataset.decode_vocab)
# Embedding层大小
EMB_SIZE = 512
# 多头数量
NHEAD = 8
# 前 馈神经网络隐藏层大小
FFN_HID_DIM = 2048
# 批 次大小
BATCH_SIZE = 8
# 编 码器层数
NUMLENCODER_LAYERS = 3
# 解码器层数
NUMLDECODER_LAYERS = 3
# 模型训练的轮数
NUML_EPOCHS = 15
# 模型训练用设备
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

In [ ]:
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
transformer = transformer.to(DEVICE)

In [ ]:
# 交叉熵损失
loss_fn = torch.nn.CrossEntropyLoss()
# Optimizer
optimizer = torch.optim.Adam(transformer.parameters(), Lr=LR)

In [ ]:
def train_epoch(train_dataloader, model, loss_fn, optimizer):
# 训练模型
    model.train()
    losses = 0
    for sre, tgt in tqdm(train_dataloader, desc='training':
    src = src.to(DEVICE)
    tgt = tgt.to(DEVICE)
    # docoder input data
    tgt_input = tgt[:-1,:]
    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
    sre_mask, tgt_mask, sre_padding_mask, tgt_padding_mask = sre_mask.to(DEVICE), tgt_mask.to(DEVICE), sre_padding_mask,to(DEVICE), tgt_padding_mask,to(DEVICE)
    togits = motel(src, tgt_input, sre_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
    optimizer.zero_grad()
    # decoder output data
    tgt_out = tgt[:-1,:]
    # logits → tbatch_size, vocab]
    loss = loss_fn(logits.reshape(-1, logits.shape[-1 ), tgt_out.reshape(-1))
    loss.backward()

    optimizer.step()
    losses += loss.item()
    
    return losses / len(train_dataloader)

In [20]:
def evaluate(val_dataloader, model, loss_fn):
#评价方法，测试集平均损失
   model.eval()
   losses = 0
   for src, tgt in tqdm(val_dataloader, desc='evaluate'):
       src = src.to(DEVICE)
       tgt = tgt.to(DEVICE)
       tgt_input =tgt[:-1, :]
       src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
       sre_mask, tgt_mask, sre_padding_mask,tgt_padding_mask = src_mask.to(DEVICE), tgt_mask.to(DEVICE), \
                                                               src_padding_mask.to(DEVICE), tgt_padding_mask.to(DEVICE)
       logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, sre_padding_mask)
       tgt_out = tgt[1:, :]
       loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
       losses += loss.item()
       return losses / len(val_dataloader)

In [21]:
# 加载编码、 解码词典
enc_vocab = load_vocab('enc.voc')
dec_vocab = load_vocab(' dec.voc')
dec_vocab_rev = { v:k for k,v in dec_vocab.items()}
# 编码器词典大小
SRCVOCAB_SIZE = len(enc_vocab)
# 解码器词典大小
TGT_VOCAB_SIZE = len(dec_vocab)
# Embedding层大小
EMB_SIZE = 512
# 多头数量
NHEAD = 8
# 前 馈神经网络隐藏层大小
FFN_HID_DIM = 2048
# 批次大小
BATCHLSIZE = 128
# 编码器层数
NUMLENCODER_LAYERS = 3
# 解码器层数
NUMLDECODER_LAYERS = 3
# 使用贪心算法生成输出序列的函数
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)
    memory = model.encode(src, src_mask)
    # decoder 输 入第一个token: BoS
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0,1 )
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob,dim=1)
        next_word = next_word.item()
        # 每 次decoder输入token是一个序列： 前 面生成出token拼接
        ys = torch.cat([ys,
                         torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == dec_vocab["EOS"]:
            break
        return ys
    def translate(model, src_sentence):
        # 输入内容编码转换
        src = torch.tensor([enc_vocab[tk] for tk in list(src_sentence)+["E0S"]]).reshape(-1, 1)
        num_tokens = src.shape[0]
        src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
        tgt_tokens = greedy_decode(
            model, src, src_mask, max_len=num_tokens + 5, start_symbol=dec_vocab["B0S"]).flatten()
        # 模型推理结果解码后还原
        return"".join([dec_vocab_rev[idx] for idx in list(tgt_tokens.cpu().numpy())]).replace("BOS”,").replace("EOS"," ")

NameError: name 'load_vocab' is not defined